## Language Processing 1: assignment 2

**Deadline: Nov 16, 23:59**

This assignment has three parts:

In the first part, you will have to analyze and extend the Minimum Edit Distance algorithm to return an alignment, and not only the minimum number of operations **(20% of the grade)**.

In the second part, you will train some language recognizers based on language models and you will be asked to find out which are the errors that the model is making **(20% of the grade)**.

The last part will be about sentiment analysis, where you will implement the Naive Bayes model that we saw in  class **(60% of the grade)**.

### Part 1: Getting an alignment using Minimum Edit Distance

In this exercise, we give you an implementation of Minimum Edit Distance.

In [1]:
import numpy as np

In [2]:
ops=['i','d','s']

def MED (src_str,trg_str):
    src_str = "#"+src_str
    trg_str = "#"+trg_str
    
    ins_cost = 1
    del_cost = 1
    sub_cost = 2
    
    m = len(src_str)
    n = len(trg_str)
    
    #INITIALIZE DISTANCE MATRIX WITH ZEROS
    distance_matrix = np.zeros((n,m))

    #INITIALIZE COLUMN 0 VALUES
    distance_matrix [:,0] = np.arange(0,n,del_cost)
    #INITIALIZE ROW 0 VALUES
    distance_matrix [0,:] = np.arange(0,m,ins_cost)    
    
    for i in range(1,n): #each column
        for j in range(1,m): #each row
            insert = distance_matrix[i-1,j] + ins_cost
            delete = distance_matrix[i,j-1] + del_cost
            if src_str[j]==trg_str[i]:
                substi = distance_matrix[i-1,j-1]
            else:
                substi = distance_matrix[i-1,j-1] + sub_cost

            distance_matrix[i,j] = min([insert,delete,substi])
            which_op = np.argmin([insert,delete,substi])
            
            print (distance_matrix[i,j],i,j, "-",insert,delete,substi, ops[which_op])
            
            
    #If you uncomment the following command you can see
    #the distance matrix in the same way
    #that appears in Figure 3.27 from the SLP2 book, page 111.
    print (np.flip(distance_matrix.T, axis=0))
    
    #RETURN THE LAST ELEMENT
    return distance_matrix[-1,-1]

MED ("PRNAP","PAP")

0.0 1 1 - 2.0 2.0 0.0 s
1.0 1 2 - 3.0 1.0 3.0 d
2.0 1 3 - 4.0 2.0 4.0 d
3.0 1 4 - 5.0 3.0 5.0 d
4.0 1 5 - 6.0 4.0 4.0 d
1.0 2 1 - 1.0 3.0 3.0 i
2.0 2 2 - 2.0 2.0 2.0 i
3.0 2 3 - 3.0 3.0 3.0 i
2.0 2 4 - 4.0 4.0 2.0 s
3.0 2 5 - 5.0 3.0 5.0 d
2.0 3 1 - 2.0 4.0 2.0 i
3.0 3 2 - 3.0 3.0 3.0 i
4.0 3 3 - 4.0 4.0 4.0 i
3.0 3 4 - 3.0 5.0 5.0 i
2.0 3 5 - 4.0 4.0 2.0 s
[[5. 4. 3. 2.]
 [4. 3. 2. 3.]
 [3. 2. 3. 4.]
 [2. 1. 2. 3.]
 [1. 0. 1. 2.]
 [0. 1. 2. 3.]]


2.0

#### Exercise 1.1 (20 pts):

You have to extend the given implementation so that it returns one alignment. Considering one minimum edit distance, there might be more than one alignment. If you return one possible alignment, the exercise will be considered correct.

The alignment should be a sequence of actions that should be applied to the source string. Considering the source string "intention" and the target string "execution", the function could return

`['d','s','s','-','i','s','-','-','-','-']`

where `d` represents deletion, `i` represents insertion, `s` represents substitution and `-` represents no change. This result could be used to represent the two aligned strings as in Figure 3.23 in SLP2.

In [3]:
ops=['i','d','s']
def MED_alignment(src_str, trg_str):
    src_str = "#"+src_str
    trg_str = "#"+trg_str
    
    ins_cost = 1
    del_cost = 1
    sub_cost = 2
    
    m = len(src_str)
    n = len(trg_str)
    
    #INITIALIZE DISTANCE MATRIX WITH ZEROS
    distance_matrix = np.zeros((n,m))

    #INITIALIZE COLUMN 0 VALUES
    distance_matrix [:,0] = np.arange(0,n,del_cost)
    #INITIALIZE ROW 0 VALUES
    distance_matrix [0,:] = np.arange(0,m,ins_cost)
    
    bck = np.zeros((n,m),dtype=str)
    
    for i in range(1,n): #each column
        for j in range(1,m): #each row
            same_character = False
            insert = distance_matrix[i-1,j] + ins_cost
            delete = distance_matrix[i,j-1] + del_cost
            if src_str[j]==trg_str[i]:
                substi = distance_matrix[i-1,j-1]
                same_character = True
            else:
                substi = distance_matrix[i-1,j-1] + sub_cost

            distance_matrix[i,j] = min([insert,delete,substi])
            which_op = np.argmin([insert,delete,substi]) 
            bck[i,j] = ops[which_op]
            
            #print (distance_matrix[i,j],i,j, "-",insert,delete,substi, ops[which_op])
        
    #YOUR CODE HERE
    
    op_seq = list() 
    # start with last cell (at the bottom right)
    i = n-1 
    j = m-1
    while i>0 or j>0: # when both i and j are smaller than 0 the loop stops
        #print(i,j)
        if i == 0: # if we are in the first row add deletion and go left in the table
            op_seq.append("d")
            j -=1
        elif j == 0: # if we are in the first row add insertion and go up in the table
            op_seq.append("i")
            i -=1
        else:
            if bck[i,j] == "s": 
                if src_str[j] == trg_str[i]: # if source and target string have the same character do nothing
                    op_seq.append("-")
                else:
                    op_seq.append("s") # otherwise add substitution to list of sequences
                i -= 1
                j -= 1
            elif bck[i,j] == "i": 
                op_seq.append("i")
                i -= 1 
            else:
                op_seq.append("d")
                j -=1
        #print(op_seq)
    op_seq = op_seq[::-1] # reverse order of operations
    
    
    
    #If you uncomment the following command you can see
    #the distance matrix in the same way
    #that appears in Figure 3.27 from the SLP2 book, page 111.
    #print (np.flip(distance_matrix.T, axis=0))
    
    #return distance_matrix,bck, op_seq_dist
    return distance_matrix[-1,-1],bck,op_seq
MED_alignment ("intention","execution")

(8.0,
 array([['', '', '', '', '', '', '', '', '', ''],
        ['', 'i', 'i', 'i', 's', 'd', 'd', 'd', 'd', 'd'],
        ['', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i'],
        ['', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i'],
        ['', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i'],
        ['', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i'],
        ['', 'i', 'i', 's', 'i', 'i', 's', 'd', 'd', 'd'],
        ['', 's', 'd', 'i', 'i', 'i', 'i', 's', 'd', 'd'],
        ['', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 's', 'd'],
        ['', 'i', 's', 'd', 'd', 'd', 'i', 'i', 'i', 's']], dtype='<U1'),
 ['d', 'd', 'd', '-', 'd', 'i', 'i', 'i', 'i', '-', '-', '-', '-'])

### Part 2: Language Models

In the following cells of code, we show you a simple language recognizer based on written bibles, available on Github (https://github.com/christos-c/bible-corpus).

Please download the whole repository (https://github.com/christos-c/bible-corpus). You can download it from the website and you should save a folder called bible-corpus.

If you prefer, you can clone the repository by executing the following cell (This command will work for those that have a Unix based system and have Git installed).

In [4]:
!git clone https://github.com/christos-c/bible-corpus.git

fatal: Zielpfad 'bible-corpus' existiert bereits und ist kein leeres Verzeichnis.


This will create a folder called bible-corpus, which contains another folder called bibles with a number of XML files. Each of these XML files contains a bible in a specific language, encoded in XML format.

The following piece of code will read some corpora. You can specify which languages you read by specifying their codes in the variable `languages`, separated by spaces. Now it loads Spanish(es), Danish(da), English(en), Basque(eu), Swedish(sv) and Finnish(fi) corpora and it saves each corpus in a dictionary called `corpus`. We can access each corpus, which is just raw text, by writing `corpus["es"]` (this will get us the spanish corpus).

In [5]:
import glob
from lxml import etree

In [6]:


languages = "da de it es en".split()


corpus = {}
for file in glob.glob("bible-corpus/bibles/*.xml"):
    print (".",end="")
    et = etree.parse(file)
    r  = et.getroot()
    
    language = r.findall("cesHeader/profileDesc/langUsage/language")[0].attrib['id']
    
    if language in languages:
        print (file)
        segs = r.findall("text/body/div/div/seg")
    
        corpus[language]  = " ".join([seg.text.strip().lower() for seg in segs if seg.text is not None])
        print(corpus[language][:100])



..................bible-corpus/bibles/Danish.xml
i begyndelsen skabte gud himmelen og jorden. og jorden var øde og tom, og der var mørke over verdens
.............bible-corpus/bibles/German.xml
am anfang schuf gott himmel und erde. und die erde war wüst und leer, und es war finster auf der tie
...................bible-corpus/bibles/English-WEB.xml
in the beginning god created the heavens and the earth. now the earth was formless and empty. darkne
.....................bible-corpus/bibles/Spanish.xml
en el principio creó dios los cielos y la tierra y la tierra estaba sin orden y vacía. había tiniebl
.......bible-corpus/bibles/Italian.xml
in principio dio creò il cielo e la terra ora la terra era informe e deserta e le tenebre ricoprivan
.............................bible-corpus/bibles/English.xml
in the beginning god created the heaven and the earth. and the earth was without form, and void; and
.

In [7]:
#Look at how the corpus starts in, e.g. English:

corpus['en'][:50]

'in the beginning god created the heaven and the ea'

In the following cell, I made a simple program (`train_lm`) to train a language model given a text. Then I included a simple function `return_proba` that gets a probability distribution and a sentence, and it returns the probability of that sentence to belong to that language.

If you want to know how this works, please feel free to check out [this simple notebook](https://absalon.ku.dk/files/7716003/download?download_frd=1) that shows how you can make a simple language recognizer.

In [8]:
import nltk
import numpy as np

In [9]:
def train_lm(data):
    # data I/O
    chars = list(set(data))
    data_size, vocab_size = len(data), len(chars)

    print ('data has %d characters, %d unique.' % (data_size, vocab_size))

    cfreq_bigrams_corpus= nltk.ConditionalFreqDist(nltk.bigrams(data))
    cprob_bigrams_corpus = nltk.ConditionalProbDist(cfreq_bigrams_corpus, nltk.MLEProbDist)
    return cprob_bigrams_corpus

def prod (ite):
    if len(ite)==0:
        return 1
    else:
        return ite[0]*prod(ite[1:])
    

def return_proba (cpd, sentence):
    return prod([cpd[bigram[0]].prob(bigram[1]) if cpd[bigram[0]].prob(bigram[1]) != 0 else 1 for bigram in nltk.bigrams(sentence)])


The code below trains a set of language models for the languages initially specified and it saves each distribution in a variable called `probability_distributions`.

In [10]:

probability_distributions = {}
for language in languages:
    print (language)
    probability_distributions[language] = train_lm(corpus[language])
print(probability_distributions["da"])

da
data has 3555654 characters, 55 unique.
de
data has 4035302 characters, 54 unique.
it
data has 3809385 characters, 56 unique.
es
data has 3912619 characters, 56 unique.
en
data has 4138511 characters, 37 unique.
<ConditionalProbDist with 55 conditions>


Here you can see a function that gets a sentence and a probability distribution as input, and it returns a list of tuples. Each tuple contains a language and a probability of belonging to that language.

In [11]:
sent = "Episoden med de mange skotske fans, der lagde et voldsomt tryk på udebanefansenes indgang til Brøndby Stadion, hvor politiet måtte trække stavene, har resulteret i én anholdelse.".lower()

def which_language(sentence, p_d):
    return [(lang, return_proba(p_d[lang],sentence)) for lang in p_d.keys()] 


In [12]:
#You can see in this case that the model is able to detect that the text is written in Danish,
#as it is the language that shows the highest probability.

which_language(sent,probability_distributions)

[('da', 1.546902713190505e-193),
 ('de', 2.4585571382154064e-208),
 ('it', 5.87111804766044e-215),
 ('es', 5.376541598888056e-210),
 ('en', 1.7619041469049169e-208)]

#### Exercise 2.1 (20 pts):

Select five languages from the corpus and train a language recognizer that can distinguish between those languages. The only thing that you have to do for this is to change the variable `languages` that on top with the languages that you desire to work with. The languages must have either the same or a similar alphabet.

Once that you selected the languages that you want to work with, you should look for (at least) five examples that the model fails to get the correct language. And you should try to justify why that is happening. Why does the model fail to guess those languages?

 * Sentence 1: The model incorrectly predicts Danish instead of German. This could be caused by the use of loanwords that do not appear in the training data ("Bug" and "gefixt"). 
 * Sentence 2: The model incorrectly predicts English instead of Italian. This could be caused by the use of propernames ("Eduardo"), numbers ("120000") and currencies ("Euro") that do not appear in the training data. 
 * Sentence 3: The model incorrectly predicts German instead of English. This could be caused by the use of informal speech.
 * Sentence 4: The model incorrectly predicts Spanish instead of German. This could be caused by the use of informal speech.
 * Sentence 5: The model incorrectly predicts Italian instead of German. This could be caused by the extensive use of medical terminology that do not appear in the training data. 

In [13]:
sentence_1 = "Ich habe gerade den Bug gefixt.".lower()
sentence_2 = "Eduardo ha vinto 120000 Euro.".lower()
sentence_3 = "Hi, what's up?".lower()
sentence_4 = "Digga, nachher chillen?".lower()
sentence_5 = "Diagnose: Paraoesophageale Hernie mit Upside-down-Magen, Therapie: Laparoskopische Hiatorhaphie und Gastrofundofrenicopexie".lower()
# https://www.juraforum.de/lexikon/fachsprache, retrieved Nov 10

print(which_language(sentence_1,probability_distributions))
print()
print(which_language(sentence_2,probability_distributions))
print()
print(which_language(sentence_3,probability_distributions))
print()
print(which_language(sentence_4,probability_distributions))
print()
print(which_language(sentence_5,probability_distributions))

[('da', 4.574665712489798e-30), ('de', 1.608947947062416e-33), ('it', 5.498477388514727e-39), ('es', 2.075284919539603e-40), ('en', 8.838954788761599e-37)]

[('da', 1.0260356470800027e-39), ('de', 8.041484617194537e-40), ('it', 1.7188974603792168e-35), ('es', 1.947968152949224e-31), ('en', 5.688092098710772e-31)]

[('da', 3.4928082009150585e-15), ('de', 1.073519587036761e-14), ('it', 2.5960233017013947e-14), ('es', 1.319101375154174e-17), ('en', 5.516637865147513e-18)]

[('da', 5.548301532493822e-33), ('de', 5.08463175589846e-29), ('it', 3.4962507507199057e-25), ('es', 3.0321337800424967e-25), ('en', 3.820820840818427e-31)]

[('da', 7.298071122060302e-167), ('de', 9.848431949474982e-166), ('it', 2.8836175799606796e-158), ('es', 1.857582972731872e-161), ('en', 8.190360763929044e-162)]


How would you improve the models so that these errors do not happen?

The model is able to capture some language specific orthographical patterns through the calculation of probability distributions of bigrams of characters. However, the model is limited in several ways.

First of all, it might pose a problem that the training data only consists of the bible. This makes it difficult to predict sentences from different genres (e.g. sentences with informal speech or terminology) or sentences containing loanwords. Increasing the training data and the variety of text genres might positively influence the accuracy of the model. 

Another shortcoming might be that the model only takes into account bigrams of characters. Looking at trigrams could be more informative as it provides more context. On the other hand, it might be too computationally expensive. Another option would be to use brigrams of tokens instead of characters. This might increase accuracy between languages with similar character patterns. 

### Part 3: Sentiment analysis

#### Naive Bayes in movie review data
 
 * Pang, B., Lee, L., & Vaithyanathan, S. (2002, July). [Thumbs up?: sentiment classification using machine learning techniques.](http://www.cs.cornell.edu/home/llee/papers/sentiment.pdf) In Proceedings of the ACL-02 conference on Empirical methods in natural language processing-Volume 10 (pp. 79-86). Association for Computational Linguistics.
 
 * You can download the data from [this website](http://www.cs.cornell.edu/people/pabo/movie-review-data/) (There are different versions, let's all download the 1.1 version of the polarity dataset (`polarity dataset v1.1 (2.2Mb) (includes README.1.1):...`))

In [14]:
import os
import numpy as np
import random

In [15]:
def filetowordlist(path, sfx):
    words = []
    for item in sorted(os.listdir(path)):
        if sfx in item:
            f=open(path + item, encoding="iso8859-1")
            lines = [line.strip() for line in f]
            f.close()
            wordsinfile = []
            for l in lines:
                sentencewords = l.split()
                wordsinfile = wordsinfile + sentencewords
            words.append(wordsinfile)
    return words

def log(number):
    return np.log(number)

##### Note: YOU MUST ADJUST THE CELL BELOW

In [16]:
#Change the directory below
#Put the directory where you downloaded the corpus

posreviews_all = filetowordlist("/Users/rea/Library/Mobile Documents/com~apple~CloudDocs/MSc_IT_and_Cognition/wintersemester2023/Language_Processing_I/Assignments/mix20_rand700_tokens_0211/tokens/pos/", ".txt")
negreviews_all = filetowordlist("/Users/rea/Library/Mobile Documents/com~apple~CloudDocs/MSc_IT_and_Cognition/wintersemester2023/Language_Processing_I/Assignments/mix20_rand700_tokens_0211/tokens/neg/", ".txt")

In [17]:
#We set 550 out of the positive reviews and 550 out of the negative reviews for creating a model
#We will teach the model how to make predictions using that portion of the data
posreviews_train = posreviews_all[:550]
negreviews_train = negreviews_all[:550]

#And later, we will be able to use the remaining part to see how well our model performs
#This is called the test data
posreviews_test  = posreviews_all[550:]
negreviews_test  = negreviews_all[550:]

In [18]:
#This is the number of positive and negative reviews in each of the portions of the data (train and test)
len(posreviews_train), len(negreviews_train),len(posreviews_test), len(negreviews_test)

(550, 550, 144, 142)

In [19]:
#These are two lists with all words in each group of reviews
#poswords_train contains a list of words, with concatenated positive reviews
poswords_train=[word for sent in posreviews_train for word in sent]
negwords_train=[word for sent in negreviews_train for word in sent]

Look at a couple of examples:

In [20]:
print (poswords_train[1000:1050])

['the', 'novels', '"', "carlito's", 'way', '"', 'and', '"', 'after', 'hours', '"', 'by', 'edwin', 'torres', ')', 'starring', ':', 'al', 'pacino', ',', 'sean', 'penn', ',', 'penelope', 'ann', 'miller', ',', 'john', 'leguiziamo', ',', 'luis', 'guzman', ',', 'john', 'rebhorn', ',', 'viggio', 'mortensen', ',', 'jorge', 'porcel', "what's", 'shocking', 'about', '"', "carlito's", 'way', '"', 'is', 'how']


In [21]:
print (negwords_train[1000:1050])

['is', 'being', 'semi-consoled', 'by', 'brooke', ',', 'one', 'of', 'his', 'graduate', 'students', '.', 'he', 'lives', 'on', 'arlington', 'road', 'where', 'he', 'makes', 'friends', 'with', 'neighbors', 'oliver', '(', 'robbins', ')', 'and', 'cheryl', 'lang', '(', 'cusack', ')', '.', 'then', 'he', 'begins', 'to', 'suspect', 'that', 'oliver', 'is', 'not', 'the', 'architect', 'he', 'claims', 'to', 'be', ',']


In [22]:
#Vocabularies for positive and negative reviews
pos_vocab_train = set(poswords_train)
neg_vocab_train = set(negwords_train)
vocab_train = pos_vocab_train.union(neg_vocab_train)

#Number of types (vocabulary size)
pos_vocab_size_train = len(pos_vocab_train)
neg_vocab_size_train = len(neg_vocab_train)
vocab_size_train = len(vocab_train)

#Number of words (tokens)
noposwords_train=len(poswords_train)
nonegwords_train=len(negwords_train)

#Number of reviews
noposreviews_train=len(posreviews_train)
nonegreviews_train=len(negreviews_train)

In [23]:
pos_vocab_size_train,noposwords_train,noposreviews_train

(27813, 445289, 550)

In [24]:
log(5),np.log10(5)

(1.6094379124341003, 0.6989700043360189)

#### Exercise 3.1 (12 pts):
Calculate prior probabilities, which are the probabilities of $P(C=positive)$ and $P(C=negative)$.

Instead of saving probabilities, remember the practical issue that we mentioned at the end of the class (we save logprobs (f.ex. `np.log`) instead of regular probabilities).

In [25]:
#YOUR CODE HERE
prior_probabiolity_pos_train = noposreviews_train/(noposreviews_train+nonegreviews_train)
prior_probabiolity_neg_train = nonegreviews_train/(noposreviews_train+nonegreviews_train)

print ("This is the log probability of a review to be positive:")
print (log(prior_probabiolity_pos_train))
print ("This is the log probability of a review to be negative:")
print (log(prior_probabiolity_neg_train))

This is the log probability of a review to be positive:
-0.6931471805599453
This is the log probability of a review to be negative:
-0.6931471805599453


##### Tricky part:

Now we need to estimate the probability of a given word, given a specific class.

$$P\left(w_{k} | c_{i}\right)=\frac{\operatorname{count}\left(w_{k}, c_{i}\right)+1}{\sum_{w \in V} \operatorname{count}\left(w, c_{i}\right) + |V|}$$

$$ logP\left(w_{k} | c_{i}\right) = log (P\left(w_{k} | c_{i}\right))$$

We need some word counts, then.

#### Exercise 3.2 (12 pts):
Calculate word counts for the positive reviews and save them in a variable.

Calculate also the word counts for the negative reviews and save them in another variable.

##### Hint: You can use a dictionary (for each document class or polarity) to save the word counts

In [26]:
#YOUR CODE HERE
from collections import Counter
pos_frequencies = Counter(poswords_train)
neg_frequencies = Counter(negwords_train)

print ("The word nice appears "+str(pos_frequencies["nice"])+" times in the positive reviews")
print ("The word nice appears "+str(neg_frequencies["nice"])+" times in the negative reviews")

print ()

print ("The word bad appears "+str(pos_frequencies["bad"])+" times in the positive reviews")
print ("The word bad appears "+str(neg_frequencies["bad"])+" times in the negative reviews")


The word nice appears 118 times in the positive reviews
The word nice appears 92 times in the negative reviews

The word bad appears 223 times in the positive reviews
The word bad appears 509 times in the negative reviews


#### Exercise 3.3 (12 pts):
Now estimate the probability of each word to appear in a positive review. Do the same with negative reviews. When you save them, do as you did in the first exercise, and save log probabilities.

##### Hint: You can use a dictionary (for each document class or polarity) to save the word logprobs

In [27]:
#YOUR CODE HERE

pos_logprobs = {key:np.log((value+1)/(vocab_size_train + noposwords_train)) for key,value in pos_frequencies.items()}
neg_logprobs = {key:np.log((value+1)/(vocab_size_train + nonegwords_train)) for key,value in neg_frequencies.items()}


print ("The log probability of the word nice to appear in the positive reviews is: "+str(pos_logprobs["nice"]))
print ("The log probability of the word nice to appear in the negative reviews is: "+str(neg_logprobs["nice"]))

print ()

print ("The log probability of the word bad to appear in the positive reviews is: "+str(pos_logprobs["bad"]))
print ("The log probability of the word bad to appear in the negative reviews is: "+str(neg_logprobs["bad"]))


The log probability of the word nice to appear in the positive reviews is: -8.310737353530826
The log probability of the word nice to appear in the negative reviews is: -8.438475770037291

The log probability of the word bad to appear in the positive reviews is: -7.678214794787315
The log probability of the word bad to appear in the negative reviews is: -6.736664537472175


#### Exercise 3.4 (12 pts):
In order to have a clean implementation, estimate the log probability of a word that doesn't appear in the training set.

This will make the code of the next exercise cleaner.

In [28]:
#YOUR CODE HERE
pos_oov_word_logprob = log(1/(vocab_size_train+noposwords_train))
neg_oov_word_logprob = log(1/(vocab_size_train+nonegwords_train)) 

print ("The log probability of an out of vocabulary word in the positive reviews is:")
print (pos_oov_word_logprob)
print ("The log probability of an out of vocabulary word in the negative reviews is:")
print (neg_oov_word_logprob)

The log probability of an out of vocabulary word in the positive reviews is:
-13.089860846642354
The log probability of an out of vocabulary word in the negative reviews is:
-12.971075263190547


#### Exercise 3.5 (12 pts):

Now we just need to write a function that will return True if an input review is positive, and False if the input review is negative.

$$\operatorname{argmax}\left(\log P\left(c_{i}\right)+\sum_{k=1}^{N} \log P\left(w_{k} | c_{i}\right)\right)$$

How would you do that?

##### Hint: Keep it simple

In [29]:
def positive_or_not(s):
    #YOUR CODE HERE
    
    prob_pos = log(prior_probabiolity_pos_train) # start with class probability for positive class
    prob_neg = log(prior_probabiolity_neg_train) # start with class probability for negative class
    
    for word in s: 
        if word in pos_logprobs: 
            prob_pos += pos_logprobs[word] # add conditional word probability to sum
        else:
            pos_oov_word_logprob 
            '''if word not in training data add log probability of a word 
            in the positive reviews that doesn't appear in the training set'''
    
    for word in s:
        if word in neg_logprobs: 
            prob_neg += neg_logprobs[word] # add conditional word probability to sum
        else:
            neg_oov_word_logprob 
            '''if word not in training data add log probability of a word in the negative 
            reviews that doesn't appear in the training set'''

    if prob_pos > prob_neg: 
        return "positive"
    else:
        return "negative"

In [30]:
review_test = "the movie is nice".split()
positive_or_not(review_test)

'positive'

In [31]:
review_test = "the movie is bad".split()
positive_or_not(review_test)

'negative'

In [32]:
review_test = "such an awful movie".split()
positive_or_not(review_test)

'negative'

In [33]:
review_test = "lovely experience".split()
positive_or_not(review_test)

'positive'